# 从 huggingface 下载 mnist 数据集

In [1]:
# 使用代理，连接 huggingface
%env ALL_PROXY=http://127.0.0.1:7890
%env HTTP_PROXY=http://127.0.0.1:7890
%env HTTPS_PROXY=http://127.0.0.1:7890

env: ALL_PROXY=http://127.0.0.1:7890
env: HTTP_PROXY=http://127.0.0.1:7890
env: HTTPS_PROXY=http://127.0.0.1:7890


In [5]:
import os
import pandas as pd

# 定义数据集的分片文件路径
splits = {'train': 'mnist/train-00000-of-00001.parquet', 
          'test': 'mnist/test-00000-of-00001.parquet'}

# 目标存储目录
output_dir = "../../data/mnist/raw/"
if not os.path.exists(output_dir):
    # 如果目录不存在，则创建目录
    os.makedirs(output_dir, exist_ok=True)

# 加载数据
train_df = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["train"])
test_df = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["test"])

# 保存数据到指定路径
train_df.to_parquet(output_dir + "train-00000-of-00001.parquet")
test_df.to_parquet(output_dir + "test-00000-of-00001.parquet")

train_df, test_df

(                                                   image  label
 0      {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      5
 1      {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      0
 2      {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      4
 3      {'bytes': b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      1
 4      {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      9
 ...                                                  ...    ...
 59995  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      8
 59996  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      3
 59997  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      5
 59998  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      6
 59999  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      8
 
 [60000 rows x 2 columns],
                                                   image  label
 0     {'bytes': b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      7
 1     {'bytes': b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      2

In [7]:
train_df.iloc[0]["image"]

{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x00\x00\x01\x00IDATx\x9cc`\x18\xcc\x80YHH\xa8\xaec\xbd\xd4\xb2\xff\xdf\xeb\x19\x18\x18\x18X`\x12rlV6\x02\xc1\x0c\x0c\x0c\x0cO&\x05~\xbex\x90\x81\x81\x81\x81\x11*g\xb8\x97\x1f\xca\xfa\x97\xf4\x95\xe1\xd9\xfb\x9b\xc8&\n\xdd\xfe\xfb\xf7\xef\xdf\xbf\xc7\xb6}\xff\x88\xc5\xbe\x809\xd9\x7f\xff\x9e\xe5f\xd0\x9e\x85\xcd5|\x8c\xb3\xfeF\xa1\n1\xc1Y\x9f\xfe\x7fdHab\xc0\x05\xb8\xf7\xfdu\xc3)\xc9\xa0\xfc\xf1\xe1\x82\x1cF\\\xb2\x81\x1f\xfe\xfe-\x97\xc4%\xab\xbb\xeb\xef\xdfi\xd2\xb8d\x05b\xff\xfc\xdd\x8d\xdb\xe2\x9f\x7f\x7f:@\x99,\xa82z!\xa6,\x0c\xd7\x0ea\xd3\xa4>\xe5\xe9\xdf\xbf\x7f\x7fm\xc3"%Qt\xf7\xef\xdf\xbf\x7fO\xfaaJ\x89;]\xfd\xfb\xf7\xef\xdfc\x81\x98\x81$\xb4\xfa\xf6\xdf\xbf\x7f\xff\x1e\x0e\xe0\xc4\x902_\xf3\xe8\xef\xdf\xbf\x7f\xbf\xb4r\xa3\x8a\xb30000\x04\x0620\\\xdf\xfc\xb7\xe7\x036WR\x1f\x00\x00E\x17^\x80kL\x00;\x00\x00\x00\x00IEND\xaeB`\x82',
 'path': None}

In [9]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
import io
import shutil

# =============== 配置信息 ===============
# Parquet 文件路径
train_parquet = "../../data/mnist/raw/train-00000-of-00001.parquet"
test_parquet = "../../data/mnist/raw/test-00000-of-00001.parquet"

# 输出目录（与您原先的 processed 目录对应）
output_root = "../../data/mnist/processed/"
train_data_dir = os.path.join(output_root, "train_data")
val_data_dir   = os.path.join(output_root, "val_data")
test_data_dir  = os.path.join(output_root, "test_data")

# 注释文件路径
train_csv = os.path.join(output_root, "train_annotations.csv")
val_csv   = os.path.join(output_root, "val_annotations.csv")
test_csv  = os.path.join(output_root, "test_annotations.csv")

# 如果这些文件夹不存在，请先创建
os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(val_data_dir, exist_ok=True)
os.makedirs(test_data_dir, exist_ok=True)

# =============== 函数定义 ===============
def save_image(image_bytes, save_path):
    """
    将二进制图像数据保存为 PNG 文件。
    """
    with open(save_path, 'wb') as f:
        f.write(image_bytes)

# =============== 读取 Parquet 文件 ===============
print("加载训练集 Parquet 文件:", train_parquet)
train_df_raw = pd.read_parquet(train_parquet)

print("加载测试集 Parquet 文件:", test_parquet)
test_df_raw = pd.read_parquet(test_parquet)

# =============== 构造训练集并划分验证集 ===============
# 假设 train_df_raw 中至少包含两列：["image", "label"]
# 为每行构造一个文件名，例如：train_0_7.png
train_df = train_df_raw.copy()
train_df["name"] = train_df.apply(
    lambda row: f"train_{row.name}_{row['label']}.png", 
    axis=1
)
train_df["name_label"] = train_df["label"]

# 使用 train_test_split 生成训练集与验证集
train_split_df, val_split_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
    stratify=train_df["name_label"]
)

# 将注释保存为 CSV
train_split_df[["name", "name_label"]].to_csv(train_csv, index=False)
val_split_df[["name", "name_label"]].to_csv(val_csv, index=False)

# =============== 将训练/验证数据保存到对应目录 ===============
print("保存训练集图像到:", train_data_dir)
for _, row in train_split_df.iterrows():
    save_path = os.path.join(train_data_dir, row["name"])
    image_bytes = row["image"]["bytes"]  # 提取二进制图像数据
    save_image(image_bytes, save_path)

print("保存验证集图像到:", val_data_dir)
for _, row in val_split_df.iterrows():
    save_path = os.path.join(val_data_dir, row["name"])
    image_bytes = row["image"]["bytes"]
    save_image(image_bytes, save_path)

# =============== 构造测试集并输出到指定目录 ===============
# 假设 test_df_raw 也有 ["image", "label"]
test_df = test_df_raw.copy()
test_df["name"] = test_df.apply(
    lambda row: f"test_{row.name}_{row['label']}.png",
    axis=1
)
test_df["name_label"] = test_df["label"]

# 保存 test_annotations.csv
test_df[["name", "name_label"]].to_csv(test_csv, index=False)

# 输出测试集图像
print("保存测试集图像到:", test_data_dir)
for _, row in test_df.iterrows():
    save_path = os.path.join(test_data_dir, row["name"])
    image_bytes = row["image"]["bytes"]
    save_image(image_bytes, save_path)

print("所有数据已成功处理并保存！")


加载训练集 Parquet 文件: ../../data/mnist/raw/train-00000-of-00001.parquet
加载测试集 Parquet 文件: ../../data/mnist/raw/test-00000-of-00001.parquet
保存训练集图像到: ../../data/mnist/processed/train_data
保存验证集图像到: ../../data/mnist/processed/val_data
保存测试集图像到: ../../data/mnist/processed/test_data
所有数据已成功处理并保存！
